In [2]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
spark_version = 'spark-3.0.3'

os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:3 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:4 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:5 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:6 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Get:7 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease [15.9 kB]
Get:8 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease [21.3 kB]
Get:9 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:10 http://security.ubuntu.com/ubuntu bionic-security/main amd64 Packages [2,694 kB]
Ign:11 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:12 http://security.ubuntu.com/ubuntu bionic-security/universe amd64 Packages [1,490 kB]
Get:13 http://security.ubuntu.com/ubunt

In [3]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2022-04-10 16:54:14--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  1.69MB/s    in 0.6s    

2022-04-10 16:54:16 (1.69 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [4]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M16-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [5]:
from pyspark import SparkFiles
from pyspark.sql.functions import to_date

url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Video_Games_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
video_game_df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
video_game_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   12039526| RTIS3L2M1F5SM|B001CXYMFS|     737716809|Thrustmaster T-Fl...|     Video Games|          5|            0|          0|   N|                Y|an amazing joysti...|Used this for Eli...| 2015-08-31|
|         US|    9636577| R1ZV7R40OLHKD|B00M920ND6|     569686175|Tonsee 6 buttons ...|     Video Games|          5|    

In [6]:
# Create the vine_table. DataFrame
vine_df = video_game_df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine", "verified_purchase"])
vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| RTIS3L2M1F5SM|          5|            0|          0|   N|                Y|
| R1ZV7R40OLHKD|          5|            0|          0|   N|                Y|
|R3BH071QLH8QMC|          1|            0|          1|   N|                Y|
|R127K9NTSXA2YH|          3|            0|          0|   N|                Y|
|R32ZWUXDJPW27Q|          4|            0|          0|   N|                Y|
|R3AQQ4YUKJWBA6|          1|            0|          0|   N|                Y|
|R2F0POU5K6F73F|          5|            0|          0|   N|                Y|
|R3VNR804HYSMR6|          5|            0|          0|   N|                Y|
| R3GZTM72WA2QH|          5|            0|          0|   N|                Y|
| RNQOY62705W1K|          4|            0|          0|   N|     

In [7]:
# Create dataframe that has reviews of 20 or more total votes.
total_votes_df = vine_df[vine_df["total_votes"] >= 20]
total_votes_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| R4PKAZRQJJX14|          1|           21|         34|   N|                N|
|R2CI0Y288CC7E2|          1|           21|         35|   N|                Y|
|R127WEQY2FM1T3|          1|          147|        175|   N|                Y|
|R3EZ0EPYLDA34S|          1|           14|         31|   N|                Y|
|R2FJ94555FZH32|          2|           55|         60|   N|                N|
|R1U3AR67RE273L|          1|           51|         65|   N|                Y|
|R3PZOXA5X1U8KW|          4|           31|         36|   N|                N|
| R6KTC1OPIOIIG|          2|           19|         34|   N|                Y|
|R36O341WWXXKNP|          5|           28|         31|   N|                N|
|R3GSK9MM8DNOYI|          1|            4|         32|   N|     

In [8]:
# Create dataframe that has reviews of 50% or more helpful votes.
helpful_votes_df = total_votes_df.filter(total_votes_df["helpful_votes"] / total_votes_df["total_votes"] >= 0.5)
helpful_votes_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| R4PKAZRQJJX14|          1|           21|         34|   N|                N|
|R2CI0Y288CC7E2|          1|           21|         35|   N|                Y|
|R127WEQY2FM1T3|          1|          147|        175|   N|                Y|
|R2FJ94555FZH32|          2|           55|         60|   N|                N|
|R1U3AR67RE273L|          1|           51|         65|   N|                Y|
|R3PZOXA5X1U8KW|          4|           31|         36|   N|                N|
| R6KTC1OPIOIIG|          2|           19|         34|   N|                Y|
|R36O341WWXXKNP|          5|           28|         31|   N|                N|
|R10LZVBLQHBVJ0|          2|          151|        198|   N|                N|
|R1VR5GLGY1GE7N|          1|           49|         51|   N|     

In [9]:
# Create dataframe that has reviews that are not a part of the vine program.
unpaid_vine_df = helpful_votes_df.filter(helpful_votes_df["vine"] == "N")
unpaid_vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| R4PKAZRQJJX14|          1|           21|         34|   N|                N|
|R2CI0Y288CC7E2|          1|           21|         35|   N|                Y|
|R127WEQY2FM1T3|          1|          147|        175|   N|                Y|
|R2FJ94555FZH32|          2|           55|         60|   N|                N|
|R1U3AR67RE273L|          1|           51|         65|   N|                Y|
|R3PZOXA5X1U8KW|          4|           31|         36|   N|                N|
| R6KTC1OPIOIIG|          2|           19|         34|   N|                Y|
|R36O341WWXXKNP|          5|           28|         31|   N|                N|
|R10LZVBLQHBVJ0|          2|          151|        198|   N|                N|
|R1VR5GLGY1GE7N|          1|           49|         51|   N|     

In [10]:
# Get the total count of non-vine reviews that have been filtered for votes.
unpaid_vine = unpaid_vine_df.count()

In [11]:
# Create dataframe that has reviews that are a part of the vine program.
paid_vine_df = helpful_votes_df.filter(helpful_votes_df["vine"] == "Y")
paid_vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R3KKUSGFZWSUIY|          5|           56|         63|   Y|                N|
|R10FO5UKKVZBK2|          3|           23|         23|   Y|                N|
| RM4KSGEOR7MU1|          5|           19|         24|   Y|                N|
| RG7VRMYLEXD23|          4|           22|         26|   Y|                N|
|R11O4YSCPSNL6L|          3|           20|         26|   Y|                N|
|R286MFBAJ8NPD6|          5|           46|         51|   Y|                N|
|R1JRR530H4COA2|          5|           22|         28|   Y|                N|
| RQ5WD90PUNBU9|          5|           21|         24|   Y|                N|
|R12648VHCQWUV9|          4|           21|         28|   Y|                N|
|R3KAW29CJ8L6DQ|          5|           17|         20|   Y|     

In [12]:
# Get the total count of vine reviews that have been filtered for votes.
paid_vine = paid_vine_df.count()

In [19]:
votes = vine_df.count()
votes

1785997

In [23]:
# Get the total count of vine reviews that have not been filtered
total_vine_reviews = vine_df[vine_df["vine"] == "Y"].count()
total_vine_reviews

4291

In [24]:
# Get the total count of non-vine reviews that have not been filtered for votes.

total_non_vine_reviews = vine_df[vine_df["vine"] == "N"].count()
total_non_vine_reviews

1781706

In [29]:
# Get the total count of 5 star vreviews that have not been filtered for votes.

total_5_reviews = vine_df[vine_df["star_rating"] == "5"].count()
total_5_reviews

1026924

In [20]:
# Get the total count of vine reviews that have been filtered for votes.

helpful_votes = helpful_votes_df.count()
helpful_votes

40565

In [32]:
# Get the total count of paid vine reviews that have been filtered for votes.

helpful_paid_review = helpful_votes_df[helpful_votes_df["vine"] == "Y"].count()
helpful_paid_review

94

In [33]:
# Get the total count of unpaid vine reviews that have been filtered for votes.

helpful_unpaid_review = helpful_votes_df[helpful_votes_df["vine"] == "N"].count()
helpful_unpaid_review

40471

In [13]:
# Get the total count of 5 star vine reviews that have been filtered for votes.

five_star = helpful_votes_df[helpful_votes_df["star_rating"] == 5].count()
five_star

15711

In [27]:
# Get the total count of 5 star vine reviews that have not been filtered for votes.

total_5_vine_reviews = vine_df.filter( (vine_df.vine == "Y") & (vine_df.star_rating == 5) ).count()
total_5_vine_reviews

1607

In [28]:
# Get the total count of 5 star non-vine reviews that have not been filtered for votes.

total_5_novine_reviews = vine_df.filter( (vine_df.vine == "N") & (vine_df.star_rating == 5) ).count()
total_5_novine_reviews

1025317

In [15]:
# Get the total count of 5 star vine reviews that have been filtered for votes.

paid_5_review = paid_vine_df[paid_vine_df["star_rating"] == 5].count()
paid_5_review

48

In [16]:
# Get the total count of 5 star non-vine reviews that have been filtered for votes.

unpaid_5_review = unpaid_vine_df[unpaid_vine_df["star_rating"] == "5"].count()
unpaid_5_review

15663

In [17]:
# Get the percentage of filtered paid 5 star reviews against filtered total 5 star reviews.
percentage_paid_5_helpful_stars = paid_5_review/ five_star
percentage_paid_5_helpful_stars

0.003055184265801031

In [30]:
# Get the percentage of non-filtered paid 5 star reviews against non-filtered total 5 star reviews.
percentage_paid_5_stars_non_helpful = total_5_vine_reviews/ total_5_reviews
percentage_paid_5_stars_non_helpful

0.0015648675072351995

In [34]:
# Get the percentage of filtered 5 unpaid star reviews against filtered unpaid total 5 star reviews.
percentage_unpaid_5_helpful_stars = unpaid_5_review / five_star
percentage_unpaid_5_helpful_stars

0.996944815734199

In [31]:
# Get the percentage of non-filtered 5 unpaid star reviews against non-filtered unpaid total 5 star reviews.

percentage_unpaid_5_stars_non_helpful = total_5_novine_reviews/ total_5_reviews
percentage_unpaid_5_stars_non_helpful

0.9984351324927648